In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Sevinj calculation of short and long term classification os_days

In [2]:
path='../../Data/'

ch_cl=pd.read_csv(path+'clinical_CHUM.csv')
chum=pd.read_csv(path+'PyRads_CHUM.csv')

new_d=pd.merge(ch_cl,chum, on='oncotech_id')


In [3]:
radiomics_features = list(chum.columns)
radiomics_features.remove("oncotech_id")
radiomics_features.remove("Unnamed: 0")

In [4]:
a=new_d.os_days.quantile(0.333)
b=new_d.os_days.quantile(0.666)
c=new_d.os_days.quantile(0.666)
print("Q1 quantile: ", new_d.os_days.quantile(0.33))
print("Q2 quantile: ", new_d.os_days.quantile(0.66))
print("Q3 quantile: ", new_d.os_days.quantile(0.66))
print(a,b,c)

df1=new_d[new_d.os_days<=a].reset_index()
df2=new_d[(new_d.os_days>a) & (new_d.os_days<=b)].reset_index()
df3=new_d[new_d.os_days>=b].reset_index()

print(len(df1), len(df3))

#rad_path_chum='/Users/sevinjyolchuyeva/Desktop/postdoc/data_os_days/data/'
column_name='class_os_days'
#with open(rad_path_chum+"feature_radimics.json", "r") as fp:
#    feature_name = json.load(fp)

radomics_first = df1
radomics_second = df3
radomics_first[column_name] = 0
radomics_second[column_name] = 1
radomics_chum=pd.concat([radomics_first, radomics_second], ignore_index=True)
radomics_chum=radomics_chum.sample(frac=1).reset_index(drop=True)

Q1 quantile:  280.78000000000003
Q2 quantile:  594.1200000000001
Q3 quantile:  594.1200000000001
282.778 602.964 602.964
74 75


# Analysis of clinical features

In [5]:
clinical_features = ["age", "sex", "ecog_status", "first_line_io", "smoking_habit", "histology_group"]
predictor = "class_os_days"

In [6]:
def change_groupstr_to_id(df, col):
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].cat.codes
    return df

In [7]:
# Data check
for clinic in clinical_features:
    if clinic in ["sex", "smoking_habit", "ecog_status", "first_line_io", "histology_group"]:
        radomics_chum = change_groupstr_to_id(radomics_chum, clinic)
    print(clinic)
    print(len(radomics_chum[radomics_chum[clinic].isna()]))


age
0
sex
0
ecog_status
0
first_line_io
0
smoking_habit
0
histology_group
0


## Univariate

In [8]:
from statsmodels.stats.multitest import fdrcorrection


In [9]:
p_val_df = pd.DataFrame(columns=["Feature", "p_val"])
for clinic in clinical_features:
    Xtrain = np.asarray(radomics_chum[[clinic]])
    ytrain = np.asarray(radomics_chum[[predictor]])
    log_reg = sm.Logit(ytrain, Xtrain).fit()
    p_temp = log_reg.pvalues[0]
    row_temp = pd.DataFrame(dict(zip(["Feature", "p_val"], [clinic, p_temp])), index=[0]) #auc_col_name  roc_score
    p_val_df = pd.concat([p_val_df,row_temp])

p_val_arr = np.array(p_val_df["p_val"])
reject, p_val_corrected = fdrcorrection(pvals=p_val_arr)
#print(p_val_corrected)
p_val_df["FDR_corr"] = p_val_corrected
p_val_df["-log10(FDR)"] = -np.log10(p_val_corrected)
p_val_df.round(5)

Optimization terminated successfully.
         Current function value: 0.693127
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.685337
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.662766
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.688818
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693122
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.687231
         Iterations 4


,Feature,p_val,FDR_corr,-log10(FDR)
0,age,0.93852,0.93852,0.02756
0,sex,0.13019,0.38434,0.41529
0,ecog_status,0.00390,0.02341,1.63057
0,first_line_io,0.26044,0.39066,0.40820
0,smoking_habit,0.93141,0.93852,0.02756
0,histology_group,0.19217,0.38434,0.41529


In [13]:
p_val_df[["Feature", "p_val"]]

,Feature,p_val
0,age,0.938520
0,sex,0.130191
0,ecog_status,0.003902
0,first_line_io,0.260440
0,smoking_habit,0.931415
0,histology_group,0.192168


## Multivariate

In [35]:
multivaris1 = ["age", "sex", "ecog_status", "first_line_io"]
multivaris2 = ["age", "sex", "ecog_status", "first_line_io",  "histology_group"]
multivaris3 = ["age", "sex", "ecog_status", "first_line_io",  "histology_group", "smoking_habit"]
multis = [multivaris1, multivaris2, multivaris3]
p_val_df_multi = pd.DataFrame(columns=["Feature", "p_val"])
p_val_df_multi_feats = pd.DataFrame(columns=["Feature", "p_val"])
def multi_analysis(variables):
    p_val_df_multi = pd.DataFrame(columns=["Feature", "p_val"])

    print(str(variables))
    Xtrain = np.asarray(radomics_chum[variables])
    ytrain = np.asarray(radomics_chum[[predictor]])
    log_reg = sm.Logit(ytrain, Xtrain).fit()

    #p_val_df_multi_feats_temp = pd.DataFrame(columns=["Feature", "p_val"])
    for i, feat in enumerate(variables):
        print(log_reg.pvalues)
        p_feat = log_reg.pvalues[i]

        row_temp = pd.DataFrame(dict(zip(["Feature", "p_val"], [str(feat), p_feat])), index=[0])
        p_val_df_multi = pd.concat([p_val_df_multi,row_temp])
    return p_val_df_multi
df1 = multi_analysis(multivaris1)

['age', 'sex', 'ecog_status', 'first_line_io']
Optimization terminated successfully.
         Current function value: 0.593273
         Iterations 5
[3.65028398e-05 5.89342872e-02 2.47011942e-05 5.75249443e-02]
[3.65028398e-05 5.89342872e-02 2.47011942e-05 5.75249443e-02]
[3.65028398e-05 5.89342872e-02 2.47011942e-05 5.75249443e-02]
[3.65028398e-05 5.89342872e-02 2.47011942e-05 5.75249443e-02]


In [36]:
df1

,Feature,p_val
0,age,0.000037
0,sex,0.058934
0,ecog_status,0.000025
0,first_line_io,0.057525


In [ ]:

list_clin_feats = ["age", "smoking_habit", "ecog_status","sex", "first_line_io", "histology_group"]#"age",
df_p_values = pd.DataFrame(columns=["Feature", "p-Value"])

X = sm.add_constant(df[list_clin_feats])
y = df.os_days
# Fit and summarize OLS model
mod = sm.OLS(y, X)
res = mod.fit()

p_temp = mod.fit().pvalues.loc[x_col]
#new_row = pd.DataFrame({"Feature": x_col, "p-Value":p_temp}, index=[0])
#df_p_values = pd.concat([df_p_values, new_row])
print(res.summary())

# scatter-plot data
#ax = df.plot(x=x_col, y='os_days', kind='scatter')
# plot regression line
#abline_plot(model_results=mod.fit(), ax=ax)
#plt.show()
#return df_p_values